In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import firedrake
from firedrake import assemble, Constant, sqrt, inner, grad, dx, ds
import icepack
from icepack.constants import ice_density as ρ_I, gravity as g

Load in the work from the previous notebook.

In [ ]:
filename = "modern_state.h5"
with firedrake.CheckpointFile(filename, "r") as chk:
    mesh = chk.load_mesh()
    C = chk.load_function(mesh, "friction")
    u_0 = chk.load_function(mesh, "velocity")

In [ ]:
coords = mesh.coordinates.dat.data_ro[:]
delta = 10e3
xmin, xmax = coords[:, 0].min() - delta, coords[:, 0].max() + delta
ymin, ymax = coords[:, 1].min() - delta, coords[:, 1].max() + delta

image_filename = icepack.datasets.fetch_mosaic_of_antarctica()
with rasterio.open(image_filename, "r") as image_file:
    height, width = image_file.height, image_file.width
    transform = image_file.transform
    window = rasterio.windows.from_bounds(
        left=xmin,
        bottom=ymin,
        right=xmax,
        top=ymax,
        transform=transform,
    )
    image = image_file.read(indexes=1, window=window, masked=True)
    
def subplots(*args, **kwargs):
    fig, axes = plt.subplots()
    axes.set_aspect("equal")
    xmin, ymin, xmax, ymax = rasterio.windows.bounds(window, transform)
    axes.imshow(
        image,
        cmap="Greys_r",
        vmin=12e3,
        vmax=16.38e3,
        extent=(xmin, xmax, ymin, ymax),
    )

    return fig, axes

In [ ]:
Q = firedrake.FunctionSpace(mesh, "CG", 2)
V = firedrake.VectorFunctionSpace(mesh, "CG", 2)

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()
thickness_filename = f"netcdf:{bedmachine_filename}:thickness"
with rasterio.open(thickness_filename, "r") as thickness_file:
    h_obs = icepack.interpolate(thickness_file, Q)
    
surface_filename = f"netcdf:{bedmachine_filename}:surface"
with rasterio.open(surface_filename, "r") as surface_file:
    s_obs = icepack.interpolate(surface_file, Q)

In [ ]:
b = firedrake.interpolate(s_obs - h_obs, Q)

In [ ]:
a = Constant(0.15)

In [ ]:
T = Constant(260.0)
A = icepack.rate_factor(T)

In [ ]:
def terminus(**kwargs):
    u = kwargs["velocity"]
    h = kwargs["thickness"]
    s = kwargs["surface"]
    h_D = kwargs["thickness_downstream"]

    τ_I = 0.5 * ρ_I * g * h**2
    τ_D = 0.5 * ρ_I * g * h_D**2

    ν = firedrake.FacetNormal(mesh)
    return (τ_I - τ_D) * inner(u, ν)

In [ ]:
model = icepack.models.IceStream(terminus=terminus)
opts = {
    "dirichlet_ids": [1, 2, 3, 5, 6, 7],
    "diagnostic_solver_type": "petsc",
    "diagnostic_solver_parameters": {
        "snes_type": "newtontr",
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
}
solver = icepack.solvers.FlowSolver(model, **opts)

Smooth over the modern thickness a bit.
No sense matching to weird oscillatory garbage.

In [ ]:
h_target = h_obs.copy(deepcopy=True)
α = Constant(2e3)
J_misfit = 0.5 * (h_target - h_obs)**2 * dx
J_penalty = 0.5 * α**2 * inner(grad(h_target), grad(h_target)) * dx
J = J_misfit + J_penalty
F = firedrake.derivative(J, h_target)
firedrake.solve(F == 0, h_target)

Let's look at the RMS average misfit between this smoothed target thickness and observations.

In [ ]:
area = Constant(assemble(Constant(1) * dx(mesh)))
rmse = np.sqrt(assemble(1 / area * (h_target - h_obs)**2 * dx))
print(f"RMS error: {rmse}")

We'll assume the thickness at the ice front has been the same for the entire duration.
This should become a control!

In [ ]:
h_D = h_target.copy(deepcopy=True)

Since we're tweaking the thickness a bit, we'll want to recompute the velocity.

In [ ]:
u_target = solver.diagnostic_solve(
    velocity=u_0,
    thickness=h_target,
    surface=s_obs,
    fluidity=A,
    friction=C,
    thickness_downstream=h_D,
)

Define the simulation we want to run.

In [ ]:
final_time = 2021.0
start_time = 1921.0
dt = 1 / 6
num_steps = int((final_time - start_time) / dt)

In [ ]:
def simulation(h_initial):
    h = h_initial.copy(deepcopy=True)

    s = icepack.compute_surface(thickness=h, bed=b)
    u = solver.diagnostic_solve(
        velocity=u_target,
        thickness=h,
        surface=s,
        fluidity=A,
        friction=C,
        thickness_downstream=h_D,
    )
    
    for step in range(num_steps):
        h = solver.prognostic_solve(
            dt,
            thickness=h,
            velocity=u,
            accumulation=a,
            thickness_inflow=h_target,
        )
    
    return h

In [ ]:
σ_h = Constant(rmse)
def loss_functional(h_final):
    return 0.5 / area * ((h_final - h_target) / σ_h)**2 * dx

In [ ]:
def regularization(h_initial):
    α = Constant(2e3)
    return 0.5 * (α / σ_h)**2 / area * inner(grad(h_initial), grad(h_initial)) * dx

In [ ]:
h_initial = h_target.copy(deepcopy=True)

In [ ]:
from icepack.statistics import (
    StatisticsProblem,
    MaximumProbabilityEstimator,
)

stats_problem = StatisticsProblem(
    simulation=simulation,
    loss_functional=loss_functional,
    regularization=regularization,
    controls=h_initial,
)

estimator = MaximumProbabilityEstimator(
    stats_problem,
    gradient_tolerance=1e-12,
    step_tolerance=5e-14,
)

In [ ]:
h_min = estimator.solve()